# G matrix calculation
## From plink files
You can directly calculate $\mathbf{G}$ from `plink` files:
```julia
using ABG
# e.g., your files are in plink.{bed,bim,fam}
G = ABG.plink_2_vr_I("plink")
```
Results are then stored in G matrix.  Be sure to put `plink` in a visible path, e.g., `~/bin`, `~/.local/bin`.
## Big M
If the genotyping platform is very dense, e.g., HD 777k for cattle, then it may be difficult to fit all genotypes at all in computer memory. 
In this case, you can calculate $\mathbf{G}$ [block by block](https://github.com/xijiang/ABG.jl/blob/master/notebooks/Big%20M.md).

# A matrix calculation
This session provides the following functions.
## Pedigree sort

```julia
sort_pedigree(pedigree::AbstractString, miss::AbstractString="0")
```
Given a pedigree file, this function will recode the pedigree to consective integers start from `1`.
Input file should have at least 3 columns.
The first 3 columns are `ID`, `sire` and `dam`.
ID in sire and dam columns are coded first.
Then the ID in ID columns are coded if their parents were already coded.
This procedure stops when all ID are coded, or a `loop`, i.e., an offspring is also its ancestor.
In the latter case, an error is reported.

By default, string `0` is for missing.  One can specify other string as missing according to your data.

A 3-column DataFrame is returnd.  
Row numbers are the recoded integers.  The 3rd column are their original names.  
The 1st and 2nd columns are their sires and dams. 

## $\mathbf{A}^{-1}$
```julia
ai = ABG.A_inverse(pedigree)
```
The better side of Julia is the results are just stored in a sparse matrix.
It can can thus deal with a pedigree of millions of ID.
The recursive C++ backend also enable it to finish the calculation fast.
E.g., it takes one/two hours to obtain a $\mathbf{A}^{-1}$ of 9-M pedigree.

## $\mathbf{F}$, inbreeding coefficients
To calculate IC of all ID in a pedigree:
```julia
ic = ABG.inbreeding_coeffcient(pedigree)
```

## $\mathbf{A}$-matrix
To calculate relationships of a `list` of ID in a `pedigree`:
```julia
_, A = ABG.A_matrix(pedigree, list)
```